In [22]:
import numpy as np
import random

# تابع ارزیابی


In [23]:

def eval_(chromo, function = "ackley"):
    
    nx = len(chromo)
    
    assert function == "ackley" or function == "beale" or function == "Booth" or function == "holder", "The Evaluation Function is incorrect"
    
    if(function == "ackley"):
        fitness = -20 * np.exp(-0.2 * np.sqrt((1/nx) * np.sum(chromo * chromo))) - np.exp((1/nx) * np.sum(np.cos(2 * np.pi * chromo))) + 20 + np.exp(1)
    
    elif(function == "beale"):
        assert len(chromo) == 2, "Beale is a 2 dimensional function"
        x = chromo[0]
        y = chromo[1]
        fitness = np.power((1.5 - x + x*y),2) + np.power((2.25 - x + x*y*y),2) + np.power((2.625 - x + x*y*y*y),2)
            
    elif(function == "Booth"):
        assert len(chromo) == 2, "2 dimensional function https://en.wikipedia.org/wiki/Test_functions_for_optimization"
        x = chromo[0]
        y = chromo[1]
        fitness = np.power(x+(2*y)-7,2)+np.power((2*x)+y-5,2)
    
    elif(function == "holder"):
        assert len(chromo) == 2, "Holder Table is a 2 dimensional function"
        x1 = chromo[0]
        x2 = chromo[1]
        fitness = -1 * np.abs(np.sin(x1) * np.cos(x2) * np.exp(np.abs(1 - (np.sqrt(x1*x1 + x2*x2))/np.pi))) 
    
    return fitness

In [24]:
print(eval_(np.array([3, 0.5]), "beale"))
print(eval_(np.array([1, 3]), "Booth"))
print(eval_(np.array([8.05502, 9.66459]), "holder"))

0.0
0
-19.208502567767606


 در زیر جمعیت به طور تصادفی ایجاد شده و ماتریسی به صورت رندم از توزیع یکنواخت با ابعاد تعداد والدین
    در تعداد نمونه ها ایجاد می شود 

# ساخت جمعیت اولیه


In [25]:

def initialize_pop(num_parents = 10, nx = 4, up = 30, low = -30):
    
    pop = np.empty([num_parents, nx])
    for i in range(0,num_parents):
        pop[i] = np.array([random.uniform(low, up) for i in range(0, nx)])

    return pop  

در درس گفته شده بود مناسب هست تعداد بچه ها 7 برابر تعداد والدین باشد 
در عملگر های لوکال همه ی ژن های فرزند از دو والد یکسان ایجاد می شود
در عملگر های گلوبال برای تعیین هر ژن فرزند از دو والد مجزا استفاده می شود 

# روش های بازترکیبی

In [26]:
def crossover_parents(parents, method = "local_uniform"):
    
    num_parents = parents.shape[0]
    nx = parents.shape[1]
    children = np.empty([num_parents*7, nx])
    
    assert method == "local_uniform" or method == "global_uniform" or method == "local_average" or method == "global_average", "The method does not exist"
    
    if method == "local_uniform":
        for i in range(0, num_parents*7):
            idx1 = random.randint(0, num_parents-1)
            idx2 = random.randint(0, num_parents-1)
            mask = [random.randint(0,1) for i in range(0, nx)]
             
            child_chromo = [parents[idx1][j] if mask[j]==0 else parents[idx2][j] for j in range(0, nx)]
            child_chromo = np.array(child_chromo)
            children[i] = child_chromo
            
    elif method == "global_uniform":
        for i in range(0, num_parents*7):
            mask = [random.randint(0,num_parents-1) for k in range(0, nx)]
            child_chromo = [parents[mask[k]][k] for k in range(0, nx)]
            child_chromo = np.array(child_chromo)
            children[i] = child_chromo
            
    elif method == "local_average":
        for i in range(0, num_parents*7):
            idx1 = random.randint(0, num_parents-1)
            idx2 = random.randint(0, num_parents-1)
            child_chromo = [(parents[idx1][j]+parents[idx2][j])/2 for j in range(0, nx)]
            child_chromo = np.array(child_chromo)
            children[i] = child_chromo
            
    elif method == "global_average":
        for i in range(0, num_parents*7):
            child_chromo = []
            for j in range(0, nx):
                idx1 = random.randint(0, num_parents-1)
                idx2 = random.randint(0, num_parents-1)
                child_chromo.append((parents[idx1][j]+parents[idx2][j])/2)
            child_chromo = np.array(child_chromo)
            children[i] = child_chromo
    
    return children

#  جهش دقیق 

In [27]:
def mutation_specific(children, max_iter, current_iter, up_bound, low_bound):
    
    sigma_max = 1
    sigma = sigma_max - (current_iter / max_iter) * sigma_max
    
    num_children = children.shape[0]
    nx = children.shape[1]
    
    for i in range(0, num_children):
        delta_x = np.random.normal(0, sigma, nx)
        children[i] = children[i] + delta_x
        for j in range(0, nx):
            if children[i][j] > up_bound:
                children[i][j] = up_bound
            elif children[i][j] < low_bound:
                children[i][j] = low_bound
  
    return children

# جهش تطبیقی

In [28]:
def mutation_adaptive(children, sigma, lower_sigma, raise_sigma, stagnation, up_bound, low_bound):
    
    if stagnation:
        sigma = 2
        for i in range(0, children.shape[0]):
            random_child = np.random.uniform(low_bound, up_bound, children.shape[1])
            children[i] = random_child
    else:
        if lower_sigma:
            sigma = sigma * 0.817
      
        elif raise_sigma:
            sigma = sigma / 0.817
    
        if sigma < 0.01:
            sigma = 0.01
        
        elif sigma > 2:
            sigma = 2
    
    num_children = children.shape[0]
    nx = children.shape[1]
    
    for i in range(0, num_children):
        delta_x = np.random.normal(0, sigma, nx)
        children[i] = children[i] + delta_x
        for j in range(0, nx):
            if children[i][j] > up_bound:
                children[i][j] = up_bound
            elif children[i][j] < low_bound:
                children[i][j] = low_bound
    
    return children, sigma

# انتخاب بازماندگان

در قسمت پایین فرزندان به تابع ارزیابی داده می شوند و در بنهایت بر اساس برازندگی خروجی 
انتخاب شده و به اندازه ی
والدین به جمعیت اصلی اضافه می شوند 

In [29]:
def select_survivors(children, method = "ackley"):
    
    num_children = children.shape[0]
    #bare inke indazeye farzandan ra balatar 7 barabar valedin gereftim
    num_pop = int(num_children / 7)
    pop = []
    fitnesses = []
    
    for i in range(0, num_children):
        fitnesses.append(eval_(children[i], method))
    
    indices = np.argsort(fitnesses)

    for i in range(0, num_pop):
        pop.append(children[indices[i]])
    pop = np.array(pop)
        
    return pop

# اجرای قسمت تابع ackley 

In [39]:
##با استفاده از جهش دقیق
max_iter = 50
upper_bound = 30
lower_bound = -30
num_parents = 10
nx = 4
function = "ackley"

best_fitness = 1000000
best_chromo = np.zeros((1, nx), dtype = "float32")

parents = initialize_pop(num_parents, nx, upper_bound, lower_bound)

for i in range(0, max_iter):
    children = crossover_parents(parents, "global_average")
    children = mutation_specific(children, max_iter, i, upper_bound, lower_bound)
    parents = select_survivors(children, function)
    
    if eval_(parents[0], function) < best_fitness:
        best_chromo = parents[0]
        best_fitness = eval_(best_chromo, function)
        iteration=i
    
    if i == 0:
        print("Iteration " + str(i+1) + ":")
        for j in range(0, parents.shape[0]):
            print(str(parents[j]) + " " + "fitness: " + str(eval_(parents[j], function)))
            
    elif i % 100 == 0:
        print("Iteration " + str(i+1) + ":")
        for j in range(0, parents.shape[0]):
            print(str(parents[j]) + " " + "fitness: " + str(eval_(parents[j], function)))
    
    elif i == max_iter-1:
        print("Iteration " + str(i+1) + ":")
        for j in range(0, parents.shape[0]):
            print(str(parents[j]) + " " + "fitness: " + str(eval_(parents[j], function)))
            
        print("THE BEST CHROMOSOME:")
        print(str(best_chromo) + " fitness: " + str(best_fitness))
        print("iteration :"+str(iteration))

Iteration 1:
[ 0.85142397 -6.23110458 -1.04424156 -5.04365881] fitness: 11.913213931005046
[ 0.55418039 -3.48955136  5.04983253  7.04071639] fitness: 13.877581355357535
[-4.81528579  4.21819595 -5.22819171  5.50075651] fitness: 14.372157752572567
[ 2.52251117 -3.02192653 -5.45678885 -8.16516949] fitness: 14.882924837764667
[ 7.50802468  0.03107553  5.12506949 10.04975584] fitness: 16.05291821427884
[-14.84574039  -1.16339192  -1.20096534   4.0239391 ] fitness: 16.650402337410892
[-11.70883512  -3.14760563   3.25608102  -6.8630487 ] fitness: 16.665136370590805
[  3.49773975  -0.53745609 -11.6153978   -7.36167261] fitness: 17.452878057686608
[-2.92641584 -4.02526558  6.14529463 16.79872693] fitness: 17.583118982708637
[-6.0311751  20.04738233 -3.14443209 -0.02044657] fitness: 17.884756390836785
Iteration 50:
[-0.00316638  0.00311875  0.00135207  0.00468418] fitness: 0.013773632477118714
[ 0.00794502 -0.00166269  0.00528477 -0.00253958] fitness: 0.02136103215479368
[-0.01600603  0.0129171

# قسمت اصلی 

In [37]:
## جهش تطبیقی

max_iter = 500
upper_bound = 10
lower_bound = -10
num_parents = 10
nx = 2
sigma = 1.5
function = "holder"

parents = initialize_pop(num_parents, nx, upper_bound, lower_bound)

fit_avg = 0
fit_avg_new = 0
lower_sigma = False
raise_sigma = False
stagnation = False

best_fitness = 1000000
best_chromo = np.zeros((1, nx), dtype = "float32")

iteration=0

for i in range(0, max_iter):
    
    
    fitnesses = []
    
    for j in range(0, parents.shape[0]):
        fitnesses.append(eval_(parents[j], function))
    fit = sum(fitnesses) / len(fitnesses)
    fit_avg_new += fit 
    
    if i % 5 == 0:
        fit_avg_new = fit_avg_new/5
        
        if (np.abs(fit_avg_new - fit_avg < 1)) and fit_avg_new > 100:
            stagnation = True
            
        if(fit_avg_new > fit_avg) and i != 5:
            lower_sigma = True
        
        elif (fit_avg_new <=0.9*fit_avg) and i != 5:
            raise_sigma = True
            
        fit_avg = fit_avg_new
        fit_avg_new = 0
        
    children = crossover_parents(parents, "global_uniform")
    children, sigma = mutation_adaptive(children, sigma, lower_sigma, raise_sigma, stagnation, upper_bound, lower_bound)
    lower_sigma = False
    raise_sigma = False
    stagnation = False
    parents = select_survivors(children, function)
    
    if eval_(parents[0], function) < best_fitness:
        best_chromo = parents[0]
        best_fitness = eval_(best_chromo, function)
        iteration=i
   
    if i == 0:
        print("Iteration " + str(i+1) + ":")
        for j in range(0, parents.shape[0]):
            print(str(parents[j]) + " " + "fitness: " + str(eval_(parents[j], function)))
            
    elif i % 100 == 0:
        print("Iteration " + str(i+1) + ":")
        for j in range(0, parents.shape[0]):
            print(str(parents[j]) + " " + "fitness: " + str(eval_(parents[j], function)))
    
    elif i == max_iter-1:
        print("Iteration " + str(i+1) + ":")
        for j in range(0, parents.shape[0]):
            print(str(parents[j]) + " " + "fitness: " + str(eval_(parents[j], function)))
            
        print(" THE BEST CHROMOSOME:")
        print(str(best_chromo) + " fitness: " + str(best_fitness))
        print("iteration :"+str(iteration))

Iteration 1:
[8.28628485 9.33171864] fitness: -17.6648287757601
[7.03395678 9.34272005] fitness: -10.34702175437425
[ 7.77757233 -6.18701068] fitness: -8.635693208663078
[-9.73935441 10.        ] fitness: -8.123553862415292
[-10.          -5.98231504] fitness: -7.8027610752780765
[-5.60415448 10.        ] fitness: -7.450341568293367
[-5.56348444  9.25049198] fitness: -7.418535367773868
[ 7.8374773  -5.77497376] fitness: -7.124908643774221
[-1.23944285  8.93185963] fitness: -5.406823124382975
[-8.35101115 -3.64262292] fitness: -5.15500163401674
Iteration 101:
[-8.06502053  9.39022213] fitness: -18.487281818126252
[-7.99992184  9.37599014] fitness: -18.381070550926978
[-7.97915444  9.94872862] fitness: -18.312257754559795
[-8.34373742  9.8322325 ] fitness: -18.070066074325357
[-8.39467316  9.68216116] fitness: -18.022535333596636
[-8.00677907 10.        ] fitness: -18.002763605421965
[-8.18019913 10.        ] fitness: -17.864075552959413
[-7.8880914 10.       ] fitness: -17.7814734789797

In [38]:
## جهش تطبیقی

max_iter = 500
upper_bound = 10
lower_bound = -10
num_parents = 3
nx = 2
sigma = 1.5
function = "Booth"

parents = initialize_pop(num_parents, nx, upper_bound, lower_bound)

fit_avg = 0
fit_avg_new = 0
lower_sigma = False
raise_sigma = False
stagnation = False

best_fitness = 1000000
best_chromo = np.zeros((1, nx), dtype = "float32")

for i in range(0, max_iter):
    
    
    fitnesses = []
    
    for j in range(0, parents.shape[0]):
        fitnesses.append(eval_(parents[j], function))
    fit = sum(fitnesses) / len(fitnesses)
    fit_avg_new += fit 
    
    if i % 5 == 0:
        fit_avg_new = fit_avg_new/5
        
        if (np.abs(fit_avg_new - fit_avg < 1)) and fit_avg_new > 100:
            stagnation = True
            
        if(fit_avg_new > fit_avg) and i != 5:
            lower_sigma = True
        
        elif (fit_avg_new <=0.9*fit_avg) and i != 5:
            raise_sigma = True
            
        fit_avg = fit_avg_new
        fit_avg_new = 0
        
    children = crossover_parents(parents, "global_uniform")
    children, sigma = mutation_adaptive(children, sigma, lower_sigma, raise_sigma, stagnation, upper_bound, lower_bound)
    lower_sigma = False
    raise_sigma = False
    stagnation = False
    parents = select_survivors(children, function)
    
    if eval_(parents[0], function) < best_fitness:
        best_chromo = parents[0]
        best_fitness = eval_(best_chromo, function)
        iteration=i
   
    if i == 0:
        print("Iteration " + str(i+1) + ":")
        for j in range(0, parents.shape[0]):
            print(str(parents[j]) + " " + "fitness: " + str(eval_(parents[j], function)))
            
    elif i % 100 == 0:
        print("Iteration " + str(i+1) + ":")
        for j in range(0, parents.shape[0]):
            print(str(parents[j]) + " " + "fitness: " + str(eval_(parents[j], function)))
    
    elif i == max_iter-1:
        print("Iteration " + str(i+1) + ":")
        for j in range(0, parents.shape[0]):
            print(str(parents[j]) + " " + "fitness: " + str(eval_(parents[j], function)))
            
        print(" THE BEST CHROMOSOME:")
        print(str(best_chromo) + " fitness: " + str(best_fitness))
        print("iteration :"+str(iteration))

Iteration 1:
[-1.28252223  6.78782095] fitness: 28.621192215804196
[-4.29848048  7.12218771] fitness: 50.6009856280962
[-4.50842426  7.08853122] fitness: 55.12321029442188
Iteration 101:
[0.1924914  3.32096906] fitness: 1.7019741444294822
[1.31237294 2.14575885] fitness: 2.001789395814573
[2.04344097 1.64284557] fitness: 3.324301755265347
Iteration 201:
[1.09330165 2.75969138] fitness: 0.15289763029247175
[0.45769865 3.48406227] fitness: 0.541974190135531
[0.42688029 3.20909002] fitness: 0.9022553073990025
Iteration 301:
[0.7140965  2.77802314] fitness: 1.1627843702915936
[2.30910764 1.38663394] fitness: 4.687005624131886
[-0.71619166  4.51154898] fitness: 5.397608646327795
Iteration 401:
[1.13166576 3.23910845] fitness: 0.6244027818065433
[-0.05816427  3.51961203] fitness: 2.5498623619105625
[-0.00911224  4.32814884] fitness: 3.189424285477073
Iteration 500:
[0.793115   3.22973241] fitness: 0.09766639842305075
[1.07969287 2.66115257] fitness: 0.3898128956196065
[0.38369474 3.64894201]